## Libraries

In [1]:
import os
import glob
import pandas as pd


## Making list of all csv files 

In [2]:
# changing our current directory where our csv files are saved
os.chdir("E:\Pandas-Sales-Analysis\SalesAnalysis\Sales_Data")

# only choosing files with .csv extension
extension = 'csv'

all_filenames = [i for i in glob.glob('*_2019.csv')]

# if we have not set this to current directory, then we have to pass all the files:
# Another way: all_filenames = [i for i in glob.glob("E:\Pandas-Sales-Analysis\SalesAnalysis\Sales_Data\January_2019.csv")]

all_filenames

['April_2019.csv',
 'August_2019.csv',
 'December_2019.csv',
 'February_2019.csv',
 'January_2019.csv',
 'July_2019.csv',
 'June_2019.csv',
 'March_2019.csv',
 'May_2019.csv',
 'November_2019.csv',
 'October_2019.csv',
 'September_2019.csv']

## Merging 12 months data into single csv file

In [5]:
# Initialising empty data frame to store 12 months data
all_months = pd.DataFrame()

x = 0
for f in all_filenames:
    df = pd.read_csv(f) 
   
    all_months = pd.concat([all_months,df])
    # pd.concat([ 1. df: where to concatenate into, 2. df which you want to add in first data frame ])

# changing a data frame to csv file using .to_csv
all_months.to_csv( "combined_data.csv", index=False)
# dataframe_name.to_csv('csv_file_name')

print("Size of merged data frame: ",all_months.shape)

all_months.sample(15)

Size of merged data frame:  (186849, 6)


,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
4055,282675,27in 4K Gaming Monitor,1,389.99,12/01/19 01:45,"501 9th St, Los Angeles, CA 90001"
607,223496,Apple Airpods Headphones,1,150,07/14/19 14:55,"7 14th St, San Francisco, CA 94016"
11501,270337,iPhone,1,700,10/07/19 11:29,"623 10th St, Boston, MA 02215"
2993,164851,ThinkPad Laptop,1,999.99,03/04/19 09:01,"738 6th St, Boston, MA 02215"
4372,299821,iPhone,1,700,12/02/19 19:13,"611 Maple St, New York City, NY 10001"
16725,275310,34in Ultrawide Monitor,1,379.99,10/16/19 21:42,"675 Hickory St, San Francisco, CA 94016"
6110,265186,USB-C Charging Cable,2,11.95,10/13/19 08:14,"146 6th St, Dallas, TX 75001"
8148,267143,iPhone,1,700,10/27/19 17:04,"939 Madison St, Los Angeles, CA 90001"
7518,243866,Macbook Pro Laptop,1,1700,08/29/19 14:18,"198 Center St, Atlanta, GA 30301"
6493,156693,Google Phone,1,600,02/26/19 19:31,"642 Center St, Seattle, WA 98101"
